In [ ]:
!pip install pandas netCDF4 -q

In [13]:
import pandas as pd
from netCDF4 import Dataset
import numpy as np


csv_file = 'pr_ma2700.csv'
df = pd.read_csv(csv_file)
netcdf_file = 'pr_ma2700.nc'
dataset = Dataset(netcdf_file, 'w', format='NETCDF4')

# Definir as dimensões
x_values = df['X'].unique()
y_values = df['Y'].unique()

dataset.createDimension('X', len(x_values))
dataset.createDimension('Y', len(y_values))

# Criar variáveis e atribuir os vaolres
x_var = dataset.createVariable('X', np.float32, ('X',))
y_var = dataset.createVariable('Y', np.float32, ('Y',))
ma_var = dataset.createVariable('MA', np.float32, ('X', 'Y',))
x_var[:] = x_values
y_var[:] = y_values

# Preencher os dados no formato
for i, row in df.iterrows():
    x_index = np.where(x_values == row['X'])[0][0]
    y_index = np.where(y_values == row['Y'])[0][0]
    ma_var[x_index, y_index] = row['MA']

# Adicionar atributos
x_var.units = 'meters'
y_var.units = 'meters'
ma_var.units = 'nT'

dataset.close()

print(f'Arquivo NetCDF criado em: {netcdf_file}')


Arquivo NetCDF criado em: pr_ma2700.nc


In [ ]:
import pandas as pd
from netCDF4 import Dataset
import numpy as np

netcdf_file = 'pr_ma2700.nc'
dataset = Dataset(netcdf_file, 'r')

x_values = dataset.variables['X'][:]
y_values = dataset.variables['Y'][:]
ma_values = dataset.variables['MA'][:, :]

# Criar uma lista de dicionários para converter em DataFrame
data = []
for i, x in enumerate(x_values):
    for j, y in enumerate(y_values):
        data.append({'X': x, 'Y': y, 'MA': ma_values[i, j]})

df = pd.DataFrame(data)

csv_file = 'pr_ma2700_teste.csv'
df.to_csv(csv_file, index=False)

dataset.close()

print(f'Arquivo CSV criado em: {csv_file}')
